In [6]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize ,
poly)
from sklearn.model_selection import train_test_split

In [5]:
from functools import partial
from sklearn.model_selection import \
(cross_validate , KFold , ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [4]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=0) # Random Seed
                                          

In [5]:
# Linear Regression
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [7]:
# Calculate the validation MSE = 23.6
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

In [9]:
# Takes in model string and returns MSE
def evalMSE(terms,
            response,
            train,
            test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train,X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test-test_pred)**2)

In [11]:
MSE = np.zeros(3)
# enumerate gives values and indices of the objects as we iterate
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower',degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
# MSE for linear, quadratic, and cubic fits
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [12]:
# The ISLP package has a wrapper called sklearn_sm()
# first arg is a model from statsmodels
# Optional args: model_str (specify formula), and model_args(dictionary of additional args)
hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model, X, Y, 
                            cv=Auto.shape[0]) # We put Auto.shape[0] for K-fold so we implemented LOOCV
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792924

In [16]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
# Calculating CV error for polynomials of degree 1-5
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M, X, Y,
                            cv = Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323827])

In [21]:
# Outer applies the operation to each pair of elemnts
A = np.array([3,5,9])
B = np.array([2,4])
np.add.outer(A,B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In [23]:
# Using K=10
cv_error = np.zeros(5)
cv = KFold(n_splits = 10,
           shuffle = True,
           random_state=0)
for i,d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,X,Y, cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720581])

In [27]:
# We can use ShuffleSplit() to implement validation set using cross_validate()
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation);
results['test_score']
                          

array([23.61661707, 22.96552529, 23.43853845, 21.72781699, 22.79416823,
       23.09191932, 23.69196999, 23.90184611, 26.53545818, 26.258467  ])

In [28]:
# Estimatine the variablility in the test error
results['test_score'].mean(), results['test_score'].std()

(23.802232661034168, 1.421845094109185)

In [31]:
# BOOTSTRAP
Portfolio = load_data('Portfolio')
# Takes in dataframe D with columns X Y, and vector idx indicating which observations should be used to est alpha
def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    # Minimum variance formula (5.7)
    return ((cov_[1,1] - cov_[0,1]) /
            (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [33]:
# Estimating alpha with all 100 observations
alpha_func(Portfolio, range(100))

0.57583207459283

In [34]:
# Randomly select 100 obs with replacement
rng = np.random.default_rng(0)
alpha_func(Portfolio,
           rng.choice(100, 100,
                      replace = True))

0.6074452469619004

In [35]:
# Computes bootstrap standard error
def boot_SE(func, D,
            n=None, B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0,0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [37]:
# Using B=1000 bootstrap replications
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE

0.09118176521277699

In [38]:
# Estimating accuracy of linear reg by assessing variability of B_0 and B_1
# This function takes in D and idx
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    # clone function copy formula to refit to new dataframe
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [40]:
# partial freezes the first two model-formula arguments of boot_OLS
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')
hp_func?
# Notice there's only the last two params in the funtion now

Signature:      hp_func(D, idx)
Call signature: hp_func(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function boot_OLS at 0x17744afc0>, ModelSpec(terms=['horsepower']), 'mpg')
File:           /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/functools.py
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.

In [42]:
rng = np.random.default_rng(0)
# 10 different bootstrap examples
np.array([hp_func(Auto,
                  rng.choice(392,
                             392,
                             replace=True)) for _ in range(10)])

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [43]:
# Compute standard errors of 1000 bootstrap estimates
hp_se = boot_SE(hp_func,
                Auto,
                B=1000,
                seed=10)
hp_se
# SE(B_0) = 0.85, SE(B_1) = 0.0073

intercept     0.848807
horsepower    0.007352
dtype: float64

In [45]:
# Formulas for SE in section 3.1.2
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se
# Notice that they're different from bootstrap bc we're estimating sigma^2 incorrectly because relationship is not entirely linear

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [46]:
quad_model = MS([poly('horsepower',2,raw=True)])
quad_func = partial(boot_OLS,
                    quad_model,
                    'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [47]:
# Compare the results to the standard errors computed using sm.OLS()
M = sm.OLS(Auto['mpg'],
           quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64